# Coolstore Zero Shot

This notebook will work with LLM to provide fixes for issues identified in [Coolstore](https://github.com/konveyor-ecosystem/coolstore) app for different kind of incidents. In this experiment, we use zero shot approach. The incidents have varying levels of complexity for the fixes. For each incident, we try to evaluate the responses under two conditions - with and without supplemental information from the analysis. We do at least 3 experiments with a prompt before concluding.

Find incidents used in this experiment [here](./analysis_output.yaml).

* [Pre-Requisites](#pre_reqs)
* [Experiments without incidents info](#without-analysis-information)
  * [Llama 13b](#e1r1_13b)
  * [Llama 70b](#e1r2_70b)
* [Experiments with incidents info](#with-analysis-information)
  * [Llama 13b](#e2r1_13b)
* [Observations](#observations)

## Pre-requisites
<a id='pre_reqs'></a>

First, we setup our environment to generate fixes with Kai for an already analyzed application. This is needed for every experiment we do in this notebook.

In a terminal, we init the Kai database by running:

```sh
make run-postgres
```

In another terminal, we load sample data into the db and run Kai server:

```sh
DROP_TABLES=true make load-data
export GENAI_KEY=<your-key>
make run-server
```

For each experiment, we either change the prompt and/or model. We restart the server everytime we have to make a change. To change the prompt, we manually update the [jinja template](../../kai/data/templates/main.jinja). 

Once Kai server comes up with a prompt and model we want, we run the following script to generate a fix:

In [ ]:
# some preliminary steps to load incidents
# to run this, use the kai venv 

import sys
import json
import requests
sys.path.append("../../../kai")
from kai.models.report import Report
from kai.routes.get_incident_solutions_for_file import (
    PostGetIncidentSolutionsForFileParams,
)

SERVER_URL = "http://0.0.0.0:8080"
APP_NAME = "coolstore"

report = Report.load_report_from_file('analysis_output.yaml')

files = report.get_impacted_files()

# for evalution, we will focus on this one file
# why this file? this has the most incidents with a mix of complexities
file = 'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java'

incidents = files[file]

file_contents = ""
with open(f"../../example/coolstore/{file}") as f:
    file_contents = f.read()

def get_params(incidents):
    return PostGetIncidentSolutionsForFileParams(
        file_name=str(file),
        file_contents=file_contents,
        application_name="coolstore",
        incidents=incidents,
        include_llm_results=True,
    )

def _generate_fix(params: PostGetIncidentSolutionsForFileParams):
    headers = {"Content-type": "application/json", "Accept": "text/plain"}
    response = requests.post(
        f"{SERVER_URL}/get_incident_solutions_for_file",
        data=params.model_dump_json(),
        headers=headers,
        timeout=3600,
    )
    return response

def generate_fix(params: PostGetIncidentSolutionsForFileParams):
    retries_left = 6
    for i in range(retries_left):
        try:
            response = _generate_fix(params)
            if response.status_code == 200:
                return response
            else:
                print(f"[{params.file_name}] received status code {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"[{params.file_name}] received exception: {e}")
            return None
        print(f"[{params.file_name}] failed to get a '200' response from the server...\
              retrying {retries_left-i} more times")
    print(f"[{params.file_name}] failed to get a '200' response from the server...\
          parameters = {params}")
    return None

def parse_response(response: requests.Response):
    try:
        result = response.json()
        if isinstance(result, str):
            return json.loads(result)
        elif isinstance(result, dict):
            return result
        else:
            print(f"unexpected response type: {type(result)}")
            print(f"response: {response}")
            sys.exit(1)
    except Exception as e:
        print(f"failed to parse response with error: {e}")
        print(f"response: {response}")
        sys.exit(1)


response = parse_response(generate_fix(get_params(incidents)))
print(response["updated_file"])

## Without analysis information
<a id='without-analysis-information'></a>

Here, we use a prompt without any information from analysis:


````
{% if model_provider.llama_header %}<s>[INST]You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.<<SYS>>{% endif %}
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

Problems in the file are referred to as incidents. Fix only the incidents described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "{{ src_file_name }}"
Source file contents:
```{{ src_file_language }}
{{ src_file_contents }}
```

## Issues

{% for incident in incidents %}
### incident {{ loop.index0 }}
Line number: {{ incident.line_number }}
{% endfor %}

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put them here.

{% if model_provider.llama_header %}[/INST]{% endif %}
````

### Responses

#### Llama 13b
<a id='e1r1_13b'></a>

Llama 13b seems to understand quite a few things about Quarkus out of the box. For instance, the annotations that are not needed in Quarkus are removed by the LLM. However, the imports are all messed up and a case of hallucination. It does understand that imports need to be updated. I think with enough information about specific imports needed, 13b can perform better.

##### Reasoning

```md
1. Update the `import` statements to use the Quarkus-specific imports.
	* Replace `import javax.ejb.*;` with `import io.quarkus.arc.*;`
	* Replace `import javax.jms.*;` with `import io.quarkus.messaging.*;`
2. Remove the `@MessageDriven` annotation as it is not needed in Quarkus.
3. Remove the `activationConfig` property as it is not needed in Quarkus.
4. Replace the `TextMessage` type with the `io.quarkus.messaging.Message` type.
5. Replace the `JMSException` type with the `IOException` type.
6. Update the `onMessage` method to use the `io.quarkus.messaging.Message` object.
7. Remove the `Transformers` class as it is not needed in Quarkus.
8. Update the `orderService` and `catalogService` injectons to use the Quarkus-specific injection mechanism.
```

##### Code

```java
package com.redhat.coolstore.service;

import io.quarkus.arc.Inject;
import io.quarkus.messaging.Message;
import io.quarkus.messaging.MessageListener;
import io.quarkus.messaging.Messages;
import io.quarkus.messaging.Topic;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.OrderTransformer;

public class OrderServiceMDB implements MessageListener<Topic<String>> {

    @Inject
    private OrderService orderService;

    @Inject
    private CatalogService catalogService;

    @Override
    public void onMessage(Message<String> rcvMessage) {
        System.out.println("Received message!");
        String orderStr = rcvMessage.getPayload();
        Order order = OrderTransformer.fromJson(orderStr);
        System.out.println("Order object is " + order);
        orderService.save(order);
        order.getItemList().forEach(item -> {
            catalogService.updateInventoryItems(item.getProductId(), item.getQuantity());
        });
    }
}
```

#### Llama 70b
<a id='e1r2_70b'></a>

70b is same as 13b in terms of knowing Quarkus specific things out of the box. However, we do not see hallucinations on imports here. In fact, it doesn't even bother to change imports despite incidents being spotted on imports. It does understand the `@Incoming` annotation though, and fixes the issue correctly. 

##### Reasoning

```md
To migrate the `OrderServiceMDB` class to Quarkus, we need to address the incidents reported. 

The incidents are related to the JavaEE annotations and imports, which are not compatible with Quarkus. 

1. The `@MessageDriven` annotation is a JavaEE annotation and needs to be replaced with a Quarkus equivalent. 
2. The `javax.ejb` imports are also JavaEE specific and need to be replaced with Quarkus imports.

Quarkus uses the `@Incoming` annotation from the `io.smallrye.reactive.messaging` package to define a message-driven bean. 

We need to replace the `@MessageDriven` annotation with `@Incoming` and update the imports accordingly.
```

##### Code

```java
package com.redhat.coolstore.service;

import javax.inject.Inject;
import javax.jms.JMSException;
import javax.jms.Message;
import javax.jms.TextMessage;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;
import io.smallrye.reactive.messaging.Incoming;

public class OrderServiceMDB {

    @Inject
    OrderService orderService;

    @Inject
    CatalogService catalogService;

    @Incoming("topic/orders")
    public void onMessage(Message rcvMessage) {
        System.out.println("\nMessage recd !");
        TextMessage msg = null;
        try {
            if (rcvMessage instanceof TextMessage) {
                msg = (TextMessage) rcvMessage;
                String orderStr = msg.getBody(String.class);
                System.out.println("Received order: " + orderStr);
                Order order = Transformers.jsonToOrder(orderStr);
                System.out.println("Order object is " + order);
                orderService.save(order);
                order.getItemList().forEach(orderItem -> {
                    catalogService.updateInventoryItems(orderItem.getProductId(), orderItem.getQuantity());
                });
            }
        } catch (JMSException e) {
            throw new RuntimeException(e);
        }
    }

}
```

## With analysis information
<a id='with-analysis-information'></a>

Here we will use a prompt that contains information generated from analysis to support:


````
{% if model_provider.llama_header %}<s>[INST]You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.<<SYS>>{% endif %}
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue (also referred to as incident) which needs to be addressed.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "{{ src_file_name }}"
Source file contents:
```{{ src_file_language }}
{{ src_file_contents }}
```

## Issues

{% for incident in incidents %}
### incident {{ loop.index0 }}
incident to fix: "{{ incident.message }}"
Line number: {{ incident.line_number }}
{% endfor %}

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

{% if model_provider.llama_header %}[/INST]{% endif %}
````

### Responses

#### Llama 13b
<a id='e2r1_13b'></a>

When specific information is provided about imports (via incident messages), unlike the previous case, 13b seems to take into consideration the imports. It does not hallucinate this time and does exactly as described in the incidents. However, it misunderstands a lot of information provided in the prompt about the annotations. It seems that it is following the incident messages verbatim and doing weird changes to annotations.

##### Reasoning

```md
1. The first step is to replace the `javax.ejb` import statement with `jakarta.ejb` as per the first incident. This is because Quarkus does not support Enterprise Java Beans (EJBs) and instead uses CDI for dependency injection.
2. Next, we need to replace the `javax.inject` import statement with `jakarta.inject` as per the second incident. This is because Quarkus uses Jakarta inject instead of Javax inject.
3. The third incident suggests replacing the `javax.jms` import statement with `jakarta.jms`. However, since we are not using any JMS features in this file, we can remove this import statement altogether.
4. The fourth incident suggests replacing the `destinationLookup` property with the `org.eclipse.microprofile.reactive.messaging.Incoming` annotation. However, since we are not using any message driven beans in this file, we can ignore this incident for now.
5. The fifth incident suggests replacing the `destinationType` property with the `jakarta.jms.Topic` class. However, since we are not using any JMS features in this file, we can ignore this incident for now.
6. The sixth incident suggests replacing the `acknowledgeMode` property with the `Auto-acknowledge` string. However, since we are not using any JMS features in this file, we can ignore this incident for now.
7. The seventh incident suggests using CDI instead of EJBs for dependency injection. Since we are not using any EJBs in this file, we can ignore this incident for now.
8. The eighth incident suggests removing the `javax.jms` import statement. Since we are not using any JMS features in this file, we can remove this import statement altogether.
9. The ninth incident suggests removing the `destinationLookup` property. Since we are not using any message driven beans in this file, we can remove this property altogether.
10. The tenth incident suggests removing the `destinationType` property. Since we are not using any JMS features in this file, we can remove this property altogether.
11. The eleventh incident suggests removing the `acknowledgeMode` property. Since we are not using any JMS features in this file, we can remove this property altogether.
```

##### Code

```java
package com.redhat.coolstore.service;

import jakarta.ejb.ActivationConfigProperty;
import jakarta.ejb.MessageDriven;
import jakarta.inject.Inject;
import jakarta.jms.JMSException;
import jakarta.jms.Message;
import jakarta.jms.MessageListener;
import jakarta.jms.TextMessage;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

@MessageDriven(name = "OrderServiceMDB", activationConfig = {
        @ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "topic/orders"),
        @ActivationConfigProperty(propertyName = "destinationType", propertyValue = "jakarta.jms.Topic"),
        @ActivationConfigProperty(propertyName = "acknowledgeMode", propertyValue = "Auto-acknowledge")
})
public class OrderServiceMDB implements MessageListener {

    @Inject
    OrderService orderService;

    @Inject
    CatalogService catalogService;

    @Override
    public void onMessage(Message rcvMessage) {
        System.out.println("Received message!");
        TextMessage msg = null;
        try {
            if (rcvMessage instanceof TextMessage) {
                msg = (TextMessage) rcvMessage;
                String orderStr = msg.getBody(String.class);
                System.out.println("Received order: " + orderStr);
                Order order = Transformers.jsonToOrder(orderStr);
                System.out.println("Order object is " + order);
                orderService.save(order);
                order.getItemList().forEach(orderItem -> {
                    catalogService.updateInventoryItems(orderItem.getProductId(), orderItem.getQuantity());
                });
            }
        } catch (JMSException e) {
            throw new RuntimeException(e);
        }
    }

}
```

#### Llama 70b

Unlike 13b, 70b is performing way better when additional analysis information is provided. It does seem to understand nuances of updating the annotations as well as imports. It seems that its taking into consideration its own Quarkus knowledge as well as incident messages. It does strike a good balance between the two.

##### Reasoning

````md
To address the incidents, we need to make the following changes:

1. Replace `javax.ejb` with `jakarta.ejb` (incidents 0 and 1): We need to update the import statements to use the Jakarta EE namespace instead of the Java EE namespace.

2. Replace `javax.inject` with `jakarta.inject` (incident 2): Similar to the previous step, we need to update the import statement to use the Jakarta EE namespace.

3. Replace `javax.jms` with `jakarta.jms` (incidents 3-6 and 11-14): We need to update the import statements to use the Jakarta EE namespace. However, we also need to consider that Quarkus uses SmallRye Reactive Messaging instead of JMS. We will need to remove the JMS-related imports and replace them with the corresponding SmallRye Reactive Messaging imports.

4. Replace `@MessageDriven` with `@ApplicationScoped` (incident 7): Since EJBs are not supported in Quarkus, we need to replace the `@MessageDriven` annotation with a CDI scope annotation like `@ApplicationScoped`.

5. Migrate `destinationLookup` property (incidents 8-10): We need to annotate the `onMessage` method with the `@Incoming` annotation from `org.eclipse.microprofile.reactive.messaging` to indicate the name of the queue.
````

##### Code

````java
package com.redhat.coolstore.service;

import jakarta.ejb.ActivationConfigProperty;
import jakarta.inject.Inject;
import jakarta.jms.JMSException;
import org.eclipse.microprofile.reactive.messaging.Incoming;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

@ApplicationScoped
public class OrderServiceMDB implements MessageListener { 

    @Inject
    OrderService orderService;

    @Inject
    CatalogService catalogService;

    @Incoming("topic/orders")
    public void onMessage(String message) {
        System.out.println("\nMessage recd !");
        String orderStr = message;
        System.out.println("Received order: " + orderStr);
        Order order = Transformers.jsonToOrder(orderStr);
        System.out.println("Order object is " + order);
        orderService.save(order);
        order.getItemList().forEach(orderItem -> {
            catalogService.updateInventoryItems(orderItem.getProductId(), orderItem.getQuantity());
        });
    }

}
````

## Observations
<a id='observations'></a>

* When no analysis information is given, Llama 13b seems to perform better than 70b in terms of fixing some Quarkus specific annotations. However, it is hallucinating on imports and importing things that don't exist. 70b on the other hand, does a comparable job on annotations. However, misses the imports altogether. One key insight is that, without supplemental information, none of the two models are good enough to get expected output.

* Llama 70b seems to provide more accurate responses compared to 13b when additional information about an incident is given. In fact, its very close to the final expected output. If analysis information provided is specific and explains a change well, 70b can fix issues to an accepteable accuracy. 

* 